In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
import numpy as np
from keras.layers import Dense, Dropout, BatchNormalization, LeakyReLU


In [6]:
file_path = '/content/Drying3.xlsx'
data = pd.read_excel(file_path, sheet_name='Sheet1')

X = data[['Drying Time (min )', 'Velocity m/s', 'Temperatures C', 'Thickness mm', 'Sample mass in (g)']]
y = data[['DR(g/min)']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
model_10_layers = Sequential([
    Dense(64, input_shape=(X_train_scaled.shape[1],)),
    BatchNormalization(),
    LeakyReLU(alpha=0.1),
    Dropout(0.2),

    Dense(128),
    BatchNormalization(),
    LeakyReLU(alpha=0.1),
    Dropout(0.2),

    Dense(64),
    BatchNormalization(),
    LeakyReLU(alpha=0.1),

    Dense(y_train.shape[1], activation='linear')
])


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


In [ ]:
reduce_lr_10 = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, min_lr=5e-7)
early_stop_10 = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)
model_10_layers.compile(optimizer=Adam(learning_rate=0.0005), loss='mse', metrics=['mae'])


In [ ]:
history = model_10_layers.fit(X_train_scaled, y_train,
                    epochs=1000,
                    batch_size=32,
                    validation_split=0.2,
                    verbose=1,
                   )

y_pred = model_10_layers.predict(X_test_scaled)

mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error (MSE):", mse)
print("Root Mean Squared Error (RMSE):", rmse)
print("Mean Absolute Error (MAE):", mae)
print("R² Score:", r2)

model_10_layers.save('final_dnn_model.keras')



Epoch 1/1000
7/7 ━━━━━━━━━━━━━━━━━━━━ 3s 48ms/step - loss: 704.5020 - mae: 20.2260 - val_loss: 481.1993 - val_mae: 15.8985
Epoch 2/1000
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 714.7975 - mae: 19.9213 - val_loss: 475.1047 - val_mae: 15.8457
Epoch 3/1000
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 702.1714 - mae: 20.5298 - val_loss: 468.4053 - val_mae: 15.7899
Epoch 4/1000
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 600.3788 - mae: 18.9274 - val_loss: 461.1197 - val_mae: 15.7420
Epoch 5/1000
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 679.6299 - mae: 21.0418 - val_loss: 453.2118 - val_mae: 15.6925
Epoch 6/1000
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 620.7354 - mae: 20.0261 - val_loss: 445.3286 - val_mae: 15.6474
Epoch 7/1000
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 602.8867 - mae: 19.8515 - val_loss: 437.6436 - val_mae: 15.6199
Epoch 8/1000
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 620.5448 - mae: 19.6946 - val_loss: 428.4957 - val_mae: 15.5671
Epoch 9/1000
7/7 ━━━━

In [10]:

model_10_layers="/content/final_dnn_model.keras"

In [16]:
import numpy as np
from keras.models import load_model
import joblib

# Load the trained model and scaler
final_dnn_model = load_model('/content/final_dnn_model.keras')
#scaler = joblib.load('path_to_your_scaler.pkl')

def predict_new(data):
    data_scaled = scaler.transform([data])
    prediction = final_dnn_model.predict(data_scaled)
    return [f"{val:.4f}" for val in prediction[0]]  # Format values to four decimal places

# New data for prediction
data_example = [210, 2, 80, 7, 488.16]
prediction = predict_new(data_example)

# Display results
print("Predictions for new data:", prediction)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 197ms/step
Predictions for new data: ['20.0108']
